# Anomalia de campo total produzida por um prisma

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html).

### Coisas para importar

In [1]:
#%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import polyprism
from fatiando.vis import mpl

### Pontos onde são calculados os dados

In [2]:
shape = (100, 100) # número de pontos ao longo de x e y
area = [-5000, 5000, -5000, 5000] # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

xc = 0.5*(area[1] + area[0])
yc = 0.5*(area[3] + area[2])

### Inclinação e a declinação do campo geomagnético local

In [3]:
inc = -30. # graus
dec = 0. # graus

In [4]:
amp = 23500.0 + 0.05*(xp - xc) + 0.08*(yp - yc) # nT

In [5]:
F = utils.ang2vec(1.0, inc, dec) # versor do campo geomagnético local

In [6]:
print F

[ 0.8660254  0.        -0.5      ]


### Modelo de prisma

In [7]:
mpl.close('all')
axis = mpl.figure().gca()
mpl.axis('scaled')
vertices = mpl.draw_polygon(area, axis, xy2ne=True)

In [8]:
print vertices

[[-4184.10041841 -4602.51046025]
 [-3974.89539749 -4001.0460251 ]
 [-3321.12970711 -3399.58158996]
 [-2013.59832636 -2902.71966527]
 [  156.90376569 -2039.74895397]
 [ 2222.80334728 -1281.38075314]
 [ 3608.78661088  -183.05439331]
 [ 4079.49790795  1202.92887029]
 [ 4105.64853556  2222.80334728]
 [ 3817.9916318   3216.52719665]
 [ 3399.58158996  3765.69037657]
 [ 2902.71966527  4001.0460251 ]
 [ 2405.85774059  3948.74476987]
 [ 2275.10460251  3556.48535565]
 [ 2745.81589958  2850.41841004]
 [ 3164.22594142  2432.0083682 ]
 [ 3164.22594142  1569.0376569 ]
 [ 2719.66527197   470.71129707]
 [ 1464.43514644  -366.10878661]
 [  470.71129707  -784.51882845]
 [-1176.77824268 -1229.07949791]
 [-2432.0083682  -1673.64016736]
 [-3504.18410042 -2248.9539749 ]
 [-4341.0041841  -2902.71966527]
 [-4733.26359833 -4027.19665272]
 [-4733.26359833 -4680.9623431 ]
 [-4471.75732218 -4759.41422594]]


In [9]:
incs = inc
decs = dec
m = 3.

model = [mesher.PolygonalPrism(vertices, 10, 1010, {'magnetization': utils.ang2vec(m, incs, decs)})]

### Anomalia de campo total (em nT)

In [10]:
bx = polyprism.bx(xp, yp, zp, model)
by = polyprism.by(xp, yp, zp, model)
bz = polyprism.bz(xp, yp, zp, model)

In [11]:
act_aprox = polyprism.tf(xp, yp, zp, model, inc, dec)

In [12]:
ct = np.sqrt((amp*F[0] + bx)**2 + (amp*F[1] + by)**2 + (amp*F[2] + bz)**2)

In [13]:
act = ct - amp

### Plot da anomalia de campo total

In [14]:
mpl.close('all')
mpl.figure(figsize=(12,10))

mpl.subplot(2,2,1)
mpl.title("Campo total (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, ct, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,2)
mpl.title("Anomalia de campo total (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,3)
mpl.title("Anomalia de campo total aproximada (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act_aprox, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,4)
mpl.title("Diferenca act_aprox - act (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act_aprox - act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.tight_layout(pad = 2.)

mpl.savefig('prisma_mag.png', dpi=300)

#mpl.show()

<img src='prisma.png', width=1200>